In [1]:
import numpy as np

In [2]:
n, p = int(12e3), int(1e4)
X = np.asfortranarray(np.random.randn(n, p))
H = X.T @ X / n

In [3]:
n, p

(12000, 10000)

In [4]:
import choosi as cs

In [5]:
beta = np.random.choice([-1,1], size=p, replace=True) * np.random.choice([3,4,5], size=p, replace=True)

In [6]:
y = X @ beta + np.random.randn(n)

In [7]:
v = -X.T @ y / n
scaling = np.ones(p)

In [8]:
x_soln = np.linalg.solve(H, -v)

In [9]:
signs = np.sign(x_soln) 

In [10]:
%%timeit
qnm = cs.optimizer.CQNMOptimizer(v, H, signs, scaling, lmda=1e-6)
x_opt = qnm.optimize(max_steps=100, tau=.5, c=.5, tol=1e-10)

3.71 s ± 268 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%%timeit
nm = cs.optimizer.NMOptimizer(v, H, signs, scaling, lmda=1e-6)
x_opt = nm.optimize(max_steps=100, tau=.5, c=.5, tol=1e-10)

2min 53s ± 4.82 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
# %%prun -s cumtime
# qnm = cs.optimizer.CQNMOptimizer(v, H, signs, scaling, lmda=1e-6)
# x_opt = qnm.optimize(max_steps=100, tau=.5, c=.5, tol=1e-10)

In [13]:
x_opt

NameError: name 'x_opt' is not defined

In [ ]:
qnm_obj = .5 * x_opt.T.dot(H.dot(x_opt)) - v.dot(x_opt)

In [ ]:
# %%prun -s cumtime
# nm = cs.optimizer.NMOptimizer(v, H, signs, scaling, lmda=1e-6)
# x_opt = nm.optimize(max_steps=100, tau=.5, c=.5, tol=1e-10)

         3822 function calls (3800 primitive calls) in 175.412 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000  175.412  175.412 {built-in method builtins.exec}
        1    0.000    0.000  175.412  175.412 <string>:1(<module>)
        1    0.000    0.000  175.412  175.412 optimizer.py:55(optimize)
        1    0.001    0.001  175.412  175.412 optimizer.py:70(_optimize)
    66/44    0.000    0.000  174.592    3.968 _interface.py:205(matvec)
       44    0.000    0.000  174.591    3.968 _interface.py:592(_matvec)
       44    0.000    0.000  174.591    3.968 optimizer.py:35(solve_matvec)
       44  174.590    3.968  174.591    3.968 linalg.py:329(solve)
       22    0.000    0.000   87.751    3.989 _interface.py:451(__matmul__)
       22    0.000    0.000   87.751    3.989 _interface.py:410(__mul__)
       22    0.000    0.000   87.751    3.989 _interface.py:419(dot)
       22    0.001    0.000 

In [ ]:
x_opt

array([-3.98290864,  4.04548157,  2.95761651, ..., -4.98096396,
        4.9821256 ,  2.98334956])

In [ ]:
nm_obj = .5 * x_opt.T.dot(H.dot(x_opt)) - v.dot(x_opt)

In [ ]:
np.allclose(qnm_obj, nm_obj)

True